<a href="https://colab.research.google.com/github/diya1094/Celebal-Technologies/blob/main/Assignment_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
file_path = "/content/drive/MyDrive/Celebal/train.csv"
df = pd.read_csv(file_path)

In [ ]:
for col in df.columns:
    if df[col].dtype in ['int64', 'float64']:
        df[col] = df[col].fillna(df[col].mean())
    else:
        df[col] = df[col].fillna(df[col].mode()[0])

#Dropped the column Id
dataset_df = df.drop('Id', axis=1)

print(dataset_df.head())
print(dataset_df.info())
print(dataset_df.isnull().sum())

   MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0          60       RL         65.0     8450   Pave  Grvl      Reg   
1          20       RL         80.0     9600   Pave  Grvl      Reg   
2          60       RL         68.0    11250   Pave  Grvl      IR1   
3          70       RL         60.0     9550   Pave  Grvl      IR1   
4          60       RL         84.0    14260   Pave  Grvl      IR1   

  LandContour Utilities LotConfig  ... PoolArea PoolQC  Fence MiscFeature  \
0         Lvl    AllPub    Inside  ...        0     Gd  MnPrv        Shed   
1         Lvl    AllPub       FR2  ...        0     Gd  MnPrv        Shed   
2         Lvl    AllPub    Inside  ...        0     Gd  MnPrv        Shed   
3         Lvl    AllPub    Corner  ...        0     Gd  MnPrv        Shed   
4         Lvl    AllPub       FR2  ...        0     Gd  MnPrv        Shed   

  MiscVal MoSold  YrSold  SaleType  SaleCondition  SalePrice  
0       0      2    2008        WD         Normal    

In [ ]:
from sklearn.model_selection import train_test_split

X = dataset_df.drop('SalePrice', axis=1)
y = dataset_df['SalePrice']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score

categorical_features = X.select_dtypes(include=['object']).columns

preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ],
    remainder='passthrough'
)

rf_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                             ('regressor', rf_model)])

gbr_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('regressor', gbr_model)])

svr_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('regressor', svr_model)])

# Train and evaluate RandomForestRegressor
rf_pipeline.fit(X_train, y_train)
rf_pred = rf_pipeline.predict(X_test)
rf_mse = mean_squared_error(y_test, rf_pred)
rf_r2 = r2_score(y_test, rf_pred)

print("RandomForestRegressor Performance:")
print(f"Mean Squared Error: {rf_mse:.2f}")
print(f"R-squared: {rf_r2:.2f}")
print("-" * 30)

# Train and evaluate GradientBoostingRegressor
gbr_pipeline.fit(X_train, y_train)
gbr_pred = gbr_pipeline.predict(X_test)
gbr_mse = mean_squared_error(y_test, gbr_pred)
gbr_r2 = r2_score(y_test, gbr_pred)

print("GradientBoostingRegressor Performance:")
print(f"Mean Squared Error: {gbr_mse:.2f}")
print(f"R-squared: {gbr_r2:.2f}")
print("-" * 30)

# Train and evaluate SVR
from sklearn.preprocessing import StandardScaler
svr_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('scaler', StandardScaler(with_mean=False)),
                              ('regressor', svr_model)])

svr_pipeline.fit(X_train, y_train)
svr_pred = svr_pipeline.predict(X_test)
svr_mse = mean_squared_error(y_test, svr_pred)
svr_r2 = r2_score(y_test, svr_pred)

print("SVR Performance:")
print(f"Mean Squared Error: {svr_mse:.2f}")
print(f"R-squared: {svr_r2:.2f}")
print("-" * 30)

RandomForestRegressor Performance:
Mean Squared Error: 849479450.94
R-squared: 0.89
------------------------------
GradientBoostingRegressor Performance:
Mean Squared Error: 709711170.35
R-squared: 0.91
------------------------------
SVR Performance:
Mean Squared Error: 7861867326.18
R-squared: -0.02
------------------------------


In [16]:
from sklearn.model_selection import GridSearchCV

param_grid_rf = {
    'regressor__n_estimators': [100, 200, 300],
    'regressor__max_depth': [None, 10, 20],
    'regressor__min_samples_split': [2, 5, 10]
}

grid_search_rf = GridSearchCV(rf_pipeline, param_grid_rf, cv=5, scoring='r2', n_jobs=-1)
grid_search_rf.fit(X_train, y_train)

print("Best parameters for RandomForestRegressor:", grid_search_rf.best_params_)
print("Best R-squared for RandomForestRegressor:", grid_search_rf.best_score_)
print("-" * 30)

param_grid_gbr = {
    'regressor__n_estimators': [100, 200, 300],
    'regressor__learning_rate': [0.01, 0.05, 0.1],
    'regressor__max_depth': [3, 5, 7]
}

grid_search_gbr = GridSearchCV(gbr_pipeline, param_grid_gbr, cv=5, scoring='r2', n_jobs=-1)
grid_search_gbr.fit(X_train, y_train)

print("Best parameters for GradientBoostingRegressor:", grid_search_gbr.best_params_)
print("Best R-squared for GradientBoostingRegressor:", grid_search_gbr.best_score_)
print("-" * 30)

Best parameters for RandomForestRegressor: {'regressor__max_depth': None, 'regressor__min_samples_split': 2, 'regressor__n_estimators': 300}
Best R-squared for RandomForestRegressor: 0.8407285403503945
------------------------------
Best parameters for GradientBoostingRegressor: {'regressor__learning_rate': 0.05, 'regressor__max_depth': 3, 'regressor__n_estimators': 300}
Best R-squared for GradientBoostingRegressor: 0.8638866750689973
------------------------------


Based on the R-squared values from the initial evaluation and the grid search results, the Gradient Boosting Regressor model performed the best among the models you trained. It had the highest R-squared score after hyperparameter tuning.


- RandomForestRegressor: Achieved an R-squared of 0.89 initially and the best R-squared after grid search was 0.84.
- GradientBoostingRegressor: Achieved an R-squared of 0.91 initially and the best R-squared after grid search was 0.86.
- SVR: Performed poorly with a negative R-squared score.
- Therefore, the GradientBoostingRegressor with the tuned hyperparameters is the best model for this task.